# Сверточные нейронные сети: Пошаговое руководство

В рамках данной лабораторной работы необходимо будет реализовать сверточные (CONV) и пулинговые (POOL) слои с использованием numpy, включая прямое и обратное распространение. 

**Нотации**:
- Надстрочный символ $[l]$ обозначает номер $l^{th}$ слоя. 
    - Например: $a^{[4]}$ - активация в $4^{th}$ слое. $W^{[5]}$ и $b^{[5]}$ - параметры в $5^{th}$ слое.

- Надстрочный символ $(i)$ обозначает $i^{th}$ пример. 
    - Например: $x^{(i)}$ - $i^{th}$ пример из обучающей выборки.
    
- Подстрочный символ $i$ обозначает $i^{th}$ номер в векторе.
    - Например: $a^{[l]}_i$ - обозначает $i^{th}$ номер активационной функции в слое $l$, если предположить что мы рассматриваем полносвязнный (FC) слой.
    
- $n_H$, $n_W$ и $n_C$ обозначает соответственно высоту, ширину и количество каналов передаваемых в текущий слой. Если есть необходимость указать конкретный слой $l$, то можно записать следующим образом: $n_H^{[l]}$, $n_W^{[l]}$, $n_C^{[l]}$. 
- $n_{H_{prev}}$, $n_{W_{prev}}$ и $n_{C_{prev}}$ обозначает высоту, ширину и количество каналов в предыдущем слое. Если есть необходимость указать конкретный слой $l$, то можно записать следующим образом: $n_H^{[l-1]}$, $n_W^{[l-1]}$, $n_C^{[l-1]}$. 

`Данный материал опирается и использует материалы курса Deep Learning от организации deeplearning.ai`
 
 Ссылка на основной курс (для желающих получить сертификаты): https://www.coursera.org/specializations/deep-learning

## 1 - Пакеты/Библиотеки

Первоначально необходимо запустить ячейку ниже, чтобы импортировать все пакеты, которые вам понадобятся во время лабораторной работы.
- [numpy](www.numpy.org) является основным пакетом для научных вычислений в Python.
- [matplotlib](http://matplotlib.org) это пакет для отрисовки графиков в Python.

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # размер графиков
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

## 2 - Описание лабораторной работы
Задачей в данной лабораторной работе явялется реализовать "строительные" блоки сверточной нейронной сети. Каждая функция, которую вы будете реализовывать, детально описана.

- Функции по реализации сверточного слоя включают:
    - Заполнение нулями
    - Окно свёртки
    - Convolution forward
    - Convolution backward
- Функции по реализации пулингово слоя включают:
    - Pooling forward
    - Создание макси (mask) 
    - Распределение значений
    - Pooling backward (optional)


<img src="images/model.png" style="width:800px;height:300px;">

**Примечания**: для каждой прямой функции есть соответствующий обратный эквивалент. Следовательно, на каждом шагу модуля прямого распространения необходимо хранить некоторые параметры в кэше. Эти параметры используются для вычисления градиентов во время обратного распространения. 

## 3 - Свёрточные нейронные сети

Созданные фреймворки помогают быстрее создать сверточный слой, но не помогают понять основы Глубокого обучения. Сверточный слой преобразует входную матрицу в выходную, но уже другого размера.

<img src="images/conv_nn.png" style="width:350px;height:200px;">

В данной части необходимо пошагово, с использованием вспомогательных функций, реализовать сверточный слой. 

### 3.1 - Zero-Padding

Zero-padding - добавляет рамку вокруг изображения с нулями:

<img src="images/PAD.png" style="width:600px;height:400px;">
<caption><center> <u> <font color='purple'> **Рисунок 1** </u><font color='purple'>  : **Zero-Padding**<br> Рисунок (3 канала, RGB) с padding - 2. </center></caption>

Ключевые преимущества padding:

- Использование слоя CONV без уменьшения размерности по высоте и ширине. Это один из важных аспектов в построении глубокой нейронной сети, так как высота/ширина будет уменьшаться по мере того, как вычисления будет продвигаться к более глубоким слоям. Важным параметром при обучении нейронной сети является - "same" свертка, в которой высота/ширина сохраняется после каждого слоя. 

- Сохраняет больше информации на краях изображения.

**Упражнение**: Реализуйте следующую функцию, которая дополняет все изображения из датасета X нулями. [Use np.pad](https://docs.scipy.org/doc/numpy/reference/generated/numpy.pad.html). Обратите внимание, если вы хотите разместить массив "a" формы $(5,5,5,5)$ с `pad = 1` для 2-го измерения, `pad = 3` для 4-го измерения и `pad = 0` для остальных, то необходимо сделаеть:
```python
a = np.pad(a, ((0,0), (1,1), (0,0), (3,3), (0,0)), mode='constant', constant_values = (0,0))
```

In [ ]:
# ОЦЕНИВАЕМОЕ: zero_pad

def zero_pad(X, pad):
    """
    Заполнение нулями все изображения из датасета X. Padding применяется к высоте и ширине изображения.
    
    Argument:
    X -- python numpy array рамером (m, n_H, n_W, n_C), который содержит в себе m изображений
    pad -- целое число, размер заполнения (padding) вокруг каждого изображения по вертикали и горизонтали.
    
    Returns:
    X_pad -- результат преобразования (m, n_H + 2*pad, n_W + 2*pad, n_C)
    """
    
    ### НАЧАЛО ВАШЕГО КОДА ЗДЕСЬ ### (≈ 1 строка кода)
    X_pad = None
    ### ОКОНЧАНИЕ ВАШЕГО КОДА ЗДЕСЬ ###
    
    return X_pad

In [ ]:
np.random.seed(1)
x = np.random.randn(4, 3, 3, 2)
x_pad = zero_pad(x, 2)
print ("x.shape =\n", x.shape)
print ("x_pad.shape =\n", x_pad.shape)
print ("x[1,1] =\n", x[1,1])
print ("x_pad[1,1] =\n", x_pad[1,1])

fig, axarr = plt.subplots(1, 2)
axarr[0].set_title('x')
axarr[0].imshow(x[0,:,:,0])
axarr[1].set_title('x_pad')
axarr[1].imshow(x_pad[0,:,:,0])

**Ожидаемый выход**:

```
x.shape =
 (4, 3, 3, 2)
x_pad.shape =
 (4, 7, 7, 2)
x[1,1] =
 [[ 0.90085595 -0.68372786]
 [-0.12289023 -0.93576943]
 [-0.26788808  0.53035547]]
x_pad[1,1] =
 [[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
```

### 3.2 - Один шаг свёртки

В данной части необходимо реализовать один шаг свёртки, в котором применить фильтр к одной позиции входа. Это будет использовано для создания conv слоя, который:

- Принимает на вход изображение 
- Применить фильтр для каждого позиции входного изображения
- На выходе возвращает другое изображение (обычно другого размера)

<img src="images/Convolution_schematic.gif" style="width:500px;height:300px;">
<caption><center> <u> <font color='purple'> **Рисунок 2** </u><font color='purple'>  : **Операция свёртки**<br>с фильтром 3x3 и шагом (stride) 1 (stride - шаг, с которым вы перемещаете окно каждый раз, когда скользите по изображению.</center></caption>

В приложениях компьютерного зрения, каждое значение матрицы слева соответствует одному пикселю, который вычисляется путём свёртки с фильтром 3x3, т.е. поэлементным перемножением каждого значения с исходной матрицией и последующим суммированием их и добавлением смещения. В данной части упражнения необходимо реализовать один шаг свёртки, который соответствует применению фильтра для каждой позиции на изображении.

**Упражнение**: Реализуйте conv_single_step(). [Hint](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.sum.html).

**Заметка**: Переменная b будет передана в виде массива numpy.  Если к массиву numpy добавить скаляр (с плавающей точкой или целое число), то в результате получится массив numpy.

In [ ]:
# ОЦЕНИВАЕМОЕ: conv_single_step

def conv_single_step(a_slice_prev, W, b):
    """
    Применение фильтра с параметрами W к срезу (a_slice_prev) выхода с функции активации предыдущего слоя.
    
    Arguments:
    a_slice_prev -- срзе входных данных с рамером (f, f, n_C_prev)
    W -- веса (Weight) размером (f, f, n_C_prev)
    b -- смещения (Bias) размером (1, 1, 1)
    
    Returns:
    Z -- скаляр (scalar), результат свёртки скользящего окна с параметрами (W, b) и среза входных данных x    
    """

    ### НАЧАЛО ВАШЕГО КОДА ЗДЕСЬ ### (≈ 2 строки кода)
    # Поэлементное умножение a_slice_prev и W. Без добавления b.
    s = None
    # Суммирование всех записей.
    Z = None
    # Добавление смещения b к Z. В результате должен получится скаляр.
    Z = None
    ### ОКОНЧАНИЕ ВАШЕГО КОДА ЗДЕСЬ ###

    return Z

In [ ]:
np.random.seed(1)
a_slice_prev = np.random.randn(4, 4, 3)
W = np.random.randn(4, 4, 3)
b = np.random.randn(1, 1, 1)

Z = conv_single_step(a_slice_prev, W, b)
print("Z =", Z)

**Ожидаемый выход**:
<table>
    <tr>
        <td>
            **Z**
        </td>
        <td>
            -6.99908945068
        </td>
    </tr>

</table>

### 3.3 - Прямое распространение

В прямом распространении необъодимо взять много различных фильтров и свернуть их с входом. Каждая "свертка" дает на выходе матрицу размером 2D. Затем необходимо сложить выходные данные в стек, чтобы получить размер - 3D: 

<center>
<video width="620" height="440" src="images/conv_kiank.mp4" type="video/mp4" controls>
</video>
</center>

**Упражнение**: 
Реализуйте функцию свёртки фильтра `W` с входом `A_prev`.  
На вход функция принимает:
* `A_prev`, выход функции активации из предыдущего слоя (для каждого батча m входов)
* Веса (weights) - `W`. Фильтр размером: `f` на `f`.
* Смещение (bias) - `b`, где каждый фильтр имеет собственный bias. 

**Подсказка**: 
1. Для выбора среза 2х2 в левом верхнем углу матрицы "a_prev" (рзмером - (5,5,3)), необходимо сделать:
```python
a_slice_prev = a_prev[0:2,0:2,:]
```
Обратите внимание на пример, в котором высота = 2, ширина = 2 и глубина = 3. Глубина - это количество каналов.  
Это будет полезно, когда необходимо определить `a_slice_prev` ниже, используя индексы `start/end`.

2. Чтобы определить a_slice, сначала нужно определить ее углы `vert_start`, `vert_end`, `horiz_start` и `horiz_end`. Эта фигура может быть полезна для того, чтобы узнать, как можно определить каждый из углов с помощью h, w, f и s в коде, приведенном ниже.

<img src="images/vert_horiz_kiank.png" style="width:400px;height:300px;">
<caption><center> <u> <font color='purple'> **Рисунок 3** </u><font color='purple'>  : **Определение среза с помощью вертикального и горизонтального start/end (с 2x2 фильтром)** <br> На этом рисунке показан только один канал.</center></caption>

**Напоминание**:
Формулы, которые соотносят выходную форму свертки с входной формой::
$$ n_H = \lfloor \frac{n_{H_{prev}} - f + 2 \times pad}{stride} \rfloor +1 $$
$$ n_W = \lfloor \frac{n_{W_{prev}} - f + 2 \times pad}{stride} \rfloor +1 $$
$$ n_C = \text{кол-во фильтров используемых для свёртки}$$

Для этого упражнения не нужно беспокоиться о векторизации, а просто реализовывать все с помощью for-loops.

#### Дополнительные подсказки

* Вы захотите использовать срезы массивов (например, `varname[0:1,:,3:5]`) для следующих переменных:  
  `a_prev_pad`, `W`, `b`.  
  Скопируйте стартовый код функции и запустите его вне определенной функции, в отдельных ячейках.  
  Проверить, что подмножество каждого массива - это размер и измерение, которое вы ожидаете.  
* Чтобы решить, как получить vert_start, vert_end; horiz_start, horiz_end, помните, что это индексы предыдущего слоя. 
  Нарисуйте пример предыдущего слоя (8 x 8, например) и текущего (выходного) слоя (2 x 2, например).  
  Индексы выходного слоя обозначаются `h` и `w`.  
* Убедитесь, что `a_slice_prev` имеет высоту, ширину и глубину.
* Помните, что `a_prev_pad` является подмножеством `A_prev_pad`.  
  Подумайте, какой из них следует использовать в циклах for.

In [ ]:
# ОЦЕНИВАЕМОЕ: conv_forward

def conv_forward(A_prev, W, b, hparameters):
    """
    Реализуйте прямое распространение для функции свёртки
    
    Arguments:
    A_prev -- выход функции активации предыдущего слоя, 
        numpy array размером - (m, n_H_prev, n_W_prev, n_C_prev)
    W -- Weights, numpy array размером (f, f, n_C_prev, n_C)
    b -- Biases, numpy array размером (1, 1, 1, n_C)
    hparameters -- python dictionary с "stride" и "pad"
        
    Returns:
    Z -- результат свёртки, numpy array размером (m, n_H, n_W, n_C)
    cache -- cache значений, необходимых для conv_backward() функции
    """
    
    ### НАЧАЛО ВАШЕГО КОДА ЗДЕСЬ ###
    # Извлечь размеры из A_prev (≈1 строка кода)  
    (m, n_H_prev, n_W_prev, n_C_prev) = None
    
    # Извлечь размеры из W (≈1 строка кода)
    (f, f, n_C_prev, n_C) = None
    
    # Извлечь информацию из "hparameters" (≈2 строки кода)
    stride = None
    pad = None
    
    # Вычислите размеры выходного объема CONV по формуле, приведенной выше. (≈2 строки кода)
    # Подсказка: используйте int() операции округления. 
    n_H = None
    n_W = None
    
    # Инициализируйте выходной значение Z нулями. (≈1 строка кода)
    Z = None
    
    # создайте A_prev_pad путём заполнения A_prev
    A_prev_pad = None
    
    for i in range(None):               # цикл по всем побучающим примерам
        a_prev_pad = None               # выбор i-го обучающего примера
        for h in range(None):           # цикл по вертикальной оси выходного параметра
            # Найти вертикальное начало и конец текущего "среза". (≈2 строки кода)
            vert_start = None
            vert_end = None
            
            for w in range(None):       # lикл по горизонтальной оси выходного параметра
                # Найти горизонтальное начало и конец текущего "среза"(≈2 строки кода)
                horiz_start = None
                horiz_end = None
                
                for c in range(None):   # цикл по всем каналам (= #filters)
                                        
                    # Используйте углы, чтобы определить срез a_prev_pad. (≈1 строка кода)
                    a_slice_prev = None
                    
                    # Свёртка с корректным фильтром W и смещением b. (≈3 строка кода)
                    weights = None
                    biases = None
                    Z[i, h, w, c] = None
                                        
    ### ОКОНЧАНИЕ ВАШЕГО КОДА ЗДЕСЬ ###
    
    # Проверка на то, что выходная форма правильная
    assert(Z.shape == (m, n_H, n_W, n_C))
    
    # Сохранение информации в "cache" для обратного распространения ошибки.
    cache = (A_prev, W, b, hparameters)
    
    return Z, cache

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(10,5,7,4)
W = np.random.randn(3,3,4,8)
b = np.random.randn(1,1,1,8)
hparameters = {"pad" : 1,
               "stride": 2}

Z, cache_conv = conv_forward(A_prev, W, b, hparameters)
print("Z's mean =\n", np.mean(Z))
print("Z[3,2,1] =\n", Z[3,2,1])
print("cache_conv[0][1][2][3] =\n", cache_conv[0][1][2][3])

**Ожидаемый выход**:
```
Z's mean =
 0.692360880758
Z[3,2,1] =
 [ -1.28912231   2.27650251   6.61941931   0.95527176   8.25132576
   2.31329639  13.00689405   2.34576051]
cache_conv[0][1][2][3] = [-1.1191154   1.9560789  -0.3264995  -1.34267579]
```

Наконец, слой CONV также должен содержать активацию, в этом случае добавлена следующая строка кода:

```python
# Получение результатов свёртки
Z[i, h, w, c] = ...
# Применение активации
A[i, h, w, c] = activation(Z[i, h, w, c])
```

## 4 - Пулинговый (Pooling) слой 

Пулинговый (POOL) слой уменьшает высоту и ширину входа. Это помогает уменьшить количество вычислений, а также помогает сделать детекторы более инвариантными к положению объекта на входе:

- Max-pooling слой: ($f, f$) выбирается максимальное значение.

- Average-pooling layer: ($f, f$) выбирается среднее значение.

<table>
<td>
<img src="images/max_pool1.png" style="width:500px;height:300px;">
<td>

<td>
<img src="images/a_pool.png" style="width:500px;height:300px;">
<td>
</table>

Пулинговые слои не имеют параметров для обратного распространения ошибки. Но они имеют гиперпараметры, такие  как размер окна $f$. Который определяет высоту и ширину $f \times f$ окна для вычисления *max* или *average*. 

### 4.1 - Pooling для прямого распространения
Далее необходимо реализовать MAX-POOL и AVG-POOL, в одной и той же функции. 

**Упражнение**: Реализуйте прямой проход. Следуйте указаниям в комментариях ниже.

**Напоминание**:
Вычисление выходного размера:

$$ n_H = \lfloor \frac{n_{H_{prev}} - f}{stride} \rfloor +1 $$

$$ n_W = \lfloor \frac{n_{W_{prev}} - f}{stride} \rfloor +1 $$

$$ n_C = n_{C_{prev}}$$

In [ ]:
# ОЦЕНИВАЕМОЕ: pool_forward

def pool_forward(A_prev, hparameters, mode = "max"):
    """
    Реализация прямого распространения для пулингового слоя
    
    Arguments:
    A_prev -- входные данные, numpy array размером (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- python dictionary, который содержит "f" и "stride"
    mode -- тип пулинга ("max" or "average")
    
    Returns:
    A -- выходное значение pool слоя, a numpy array размером (m, n_H, n_W, n_C)
    cache -- кэш, используемый в обратном проходе слоя пула, который содержит входы и гиперпараметры  
    """
    
    # Входной размер
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Извлечение гиперпараметров "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]
    
    # Определите размеры выходного слоя
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    # Инициализация выходной матрицы A
    A = np.zeros((m, n_H, n_W, n_C))              
    
    ### НАЧАЛО ВАШЕГО КОДА ЗДЕСЬ ###
    for i in range(None):                         # цико по обучающим примерам
        for h in range(None):                     # цикл по вертикальной оси выходного параметра
            vert_start = None
            vert_end = None
            
            for w in range(None):                 # цикл по горизонтальной оси выходного параметра
                horiz_start = None
                horiz_end = None
                
                for c in range (None):            # цикл по каналам выходного параметра
                    
                    # Используйте углы, чтобы определить текущий срез на i-м обучающем примере A_prev, канал c. (≈1 строка)
                    a_prev_slice = None
                    
                    # Провести вычисление операции объединения.
                    # Используйте np.max и np.mean.
                    if mode == "max":
                        A[i, h, w, c] = None
                    elif mode == "average":
                        A[i, h, w, c] = None
    
    ### ОКОНЧАНИЕ ВАШЕГО КОДА ЗДЕСЬ ###
    
    # Сохранение параметров в "cache" для pool_backward()
    cache = (A_prev, hparameters)
    
    # Проверка корректности размерности
    assert(A.shape == (m, n_H, n_W, n_C))
    
    return A, cache

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(2, 5, 5, 3)
hparameters = {"stride" : 1, "f": 3}

A, cache = pool_forward(A_prev, hparameters)
print("mode = max")
print("A.shape = " + str(A.shape))
print("A =\n", A)
print()
A, cache = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A.shape = " + str(A.shape))
print("A =\n", A)

**Ожидаемый выход**
```
mode = max
A.shape = (2, 3, 3, 3)
A =
 [[[[ 1.74481176  0.90159072  1.65980218]
   [ 1.74481176  1.46210794  1.65980218]
   [ 1.74481176  1.6924546   1.65980218]]

  [[ 1.14472371  0.90159072  2.10025514]
   [ 1.14472371  0.90159072  1.65980218]
   [ 1.14472371  1.6924546   1.65980218]]

  [[ 1.13162939  1.51981682  2.18557541]
   [ 1.13162939  1.51981682  2.18557541]
   [ 1.13162939  1.6924546   2.18557541]]]


 [[[ 1.19891788  0.84616065  0.82797464]
   [ 0.69803203  0.84616065  1.2245077 ]
   [ 0.69803203  1.12141771  1.2245077 ]]

  [[ 1.96710175  0.84616065  1.27375593]
   [ 1.96710175  0.84616065  1.23616403]
   [ 1.62765075  1.12141771  1.2245077 ]]

  [[ 1.96710175  0.86888616  1.27375593]
   [ 1.96710175  0.86888616  1.23616403]
   [ 1.62765075  1.12141771  0.79280687]]]]

mode = average
A.shape = (2, 3, 3, 3)
A =
 [[[[ -3.01046719e-02  -3.24021315e-03  -3.36298859e-01]
   [  1.43310483e-01   1.93146751e-01  -4.44905196e-01]
   [  1.28934436e-01   2.22428468e-01   1.25067597e-01]]

  [[ -3.81801899e-01   1.59993515e-02   1.70562706e-01]
   [  4.73707165e-02   2.59244658e-02   9.20338402e-02]
   [  3.97048605e-02   1.57189094e-01   3.45302489e-01]]

  [[ -3.82680519e-01   2.32579951e-01   6.25997903e-01]
   [ -2.47157416e-01  -3.48524998e-04   3.50539717e-01]
   [ -9.52551510e-02   2.68511000e-01   4.66056368e-01]]]


 [[[ -1.73134159e-01   3.23771981e-01  -3.43175716e-01]
   [  3.80634669e-02   7.26706274e-02  -2.30268958e-01]
   [  2.03009393e-02   1.41414785e-01  -1.23158476e-02]]

  [[  4.44976963e-01  -2.61694592e-03  -3.10403073e-01]
   [  5.08114737e-01  -2.34937338e-01  -2.39611830e-01]
   [  1.18726772e-01   1.72552294e-01  -2.21121966e-01]]

  [[  4.29449255e-01   8.44699612e-02  -2.72909051e-01]
   [  6.76351685e-01  -1.20138225e-01  -2.44076712e-01]
   [  1.50774518e-01   2.89111751e-01   1.23238536e-03]]]]
```

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(2, 5, 5, 3)
hparameters = {"stride" : 2, "f": 3}

A, cache = pool_forward(A_prev, hparameters)
print("mode = max")
print("A.shape = " + str(A.shape))
print("A =\n", A)
print()

A, cache = pool_forward(A_prev, hparameters, mode = "average")
print("mode = average")
print("A.shape = " + str(A.shape))
print("A =\n", A)

**Ожидаемый выход:**
    
```
mode = max
A.shape = (2, 2, 2, 3)
A =
 [[[[ 1.74481176  0.90159072  1.65980218]
   [ 1.74481176  1.6924546   1.65980218]]

  [[ 1.13162939  1.51981682  2.18557541]
   [ 1.13162939  1.6924546   2.18557541]]]


 [[[ 1.19891788  0.84616065  0.82797464]
   [ 0.69803203  1.12141771  1.2245077 ]]

  [[ 1.96710175  0.86888616  1.27375593]
   [ 1.62765075  1.12141771  0.79280687]]]]

mode = average
A.shape = (2, 2, 2, 3)
A =
 [[[[-0.03010467 -0.00324021 -0.33629886]
   [ 0.12893444  0.22242847  0.1250676 ]]

  [[-0.38268052  0.23257995  0.6259979 ]
   [-0.09525515  0.268511    0.46605637]]]


 [[[-0.17313416  0.32377198 -0.34317572]
   [ 0.02030094  0.14141479 -0.01231585]]

  [[ 0.42944926  0.08446996 -0.27290905]
   [ 0.15077452  0.28911175  0.00123239]]]]
```

## 5 - Обратное распространение по нейронной сети


В современных фреймоворках глубокого обучения достаточно реализовать прямой проход, а фреймоворки заботятся о обратном проходе, поэтому большинству инженеров-специалистов в области глубокого обучения не нужно беспокоиться о деталях обратного прохода. Обратный проход для свёрточных сетей сложен.  

Когда в предыдущем курсе вы реализовали простую (полностью подключенную) нейронную сеть, вы использовали обратное распространения для вычисления производных относительно функции потерь. Аналогично, в свёрточных нейронных сетях можно вычислить производные относительно функции потерь для обновления параметров.

### 5.1 - Свёртка для обратного распространения

#### 5.1.1 - Вычисление dA:
Это формула вычисления $dA$ относительно стоимости определенного фильтра $W_c$:

$$ dA += \sum _{h=0} ^{n_H} \sum_{w=0} ^{n_W} W_c \times dZ_{hw} \tag{1}$$

где $W_c$ - фильтр, а $dZ_{hw}$ - скаляр, соответствующий градиенту стоимости относительно вывода слоя Z в h-й строке и w-му столбце (соответствует точечному произведению, взятому при i-м шаге влево и j-м шаге вниз). Обратите внимание, что каждый раз при обновлении dA мы умножаем один и тот же фильтр $W_c$ на разные dZ. Мы делаем это в основном потому, что при вычислении прямого распространения, каждый фильтр точечно суммируется разными a_slice. Поэтому при вычислении обратного распространения для dA мы просто добавляем градиенты всех a_slices.  

В коде, внутри соответствующих for-loops:
```python
da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += W[:,:,:,c] * dZ[i, h, w, c]
```

#### 5.1.2 - Вычисление dW:
Это формула вычисляет $dW_c$ ($dW_c$ - производная одного фильтра) относительно функции потерь:

$$ dW_c  += \sum _{h=0} ^{n_H} \sum_{w=0} ^ {n_W} a_{slice} \times dZ_{hw}  \tag{2}$$

где $a_{slice}$ соответствует срезу, который использовался для активации $Z_{ij}$. Таким образом, получается градиент для $W$ по отношению к этому срезу. Поскольку это те же самые $W$, необходимо сложить все такие градиенты, чтобы получить $dW$.

```python
dW[:,:,:,c] += a_slice * dZ[i, h, w, c]
```

#### 5.1.3 - Вычисление db:

Это формула вычисляет $db$ в отношении потерь для определённого фильтра $W_c$:

$$ db = \sum_h \sum_w dZ_{hw} \tag{3}$$

В базовых нейронных сетях, db вычисляется суммированием $dZ$. В этом случае просто суммируются все градиенты вывода conv (Z) относительно стоимости. 

```python
db[:,:,:,c] += dZ[i, h, w, c]
```

**Упражнение**: Реализуйте `conv_backward`. Необходимо суммировать все примеры обучения, фильтры, высоту и ширину. Затем следует вычислить производные по формулам 1, 2 и 3, приведенным выше. 

In [14]:
def conv_backward(dZ, cache):
    """
    Реализация обратного распространения для свёртки
    
    Arguments:
    dZ -- градиент по потерям относительно вывода слоя conv (Z), array numpy разер (m, n_H, n_W, n_C)
    cache -- cache значений для вычисления conv_backward(), выход функции conv_forward()
    
    Returns:
    dA_prev -- градиент стоимости относительно ввода слоя свертки (A_prev),
               numpy array размером (m, n_H_prev, n_W_prev, n_C_prev)                
    dW -- градиент стоимости относительно веса conv слоя (W)
          numpy array размером (f, f, n_C_prev, n_C)
    db -- градиент стоимости относительно biases по conv слою (b)
          numpy array размером (1, 1, 1, n_C)
    """
    
    ### НАЧАЛО ВАШЕГО КОДА ЗДЕСЬ ###
    # Извлечение "cache"
    (A_prev, W, b, hparameters) = None
    
    # Извлечение размера из A_prev
    (m, n_H_prev, n_W_prev, n_C_prev) = None
    
    # Извлечение размера из W
    (f, f, n_C_prev, n_C) = None
    
    # Извлечение информации из "hparameters"
    stride = None
    pad = None
    
    # Извлечение размера из dZ
    (m, n_H, n_W, n_C) = None
    
    # Иницализация dA_prev, dW, db с корректными размерами
    dA_prev = None                           
    dW = None
    db = None

    # Заполнение A_prev и dA_prev
    A_prev_pad = None
    dA_prev_pad = None
    
    for i in range(None):                       # цикл по обучающим примерам
        
        # выбор i-го обучающего примера из A_prev_pad и dA_prev_pad
        a_prev_pad = None
        da_prev_pad = None
        
        for h in range(None):                   # цикл по вертикальной оси
            for w in range(None):               # цикл по горизонтальной оси
                for c in range(None):           # цикл по каналам                    
                    vert_start = None
                    vert_end = None
                    horiz_start = None
                    horiz_end = None
                    
                    # Используйте углы, чтобы определить срез из a_prev_pad.
                    a_slice = None

                    # Обновление градиентов для параметров фильтра с помощью приведенных выше формул
                    da_prev_pad[vert_start:vert_end, horiz_start:horiz_end, :] += None
                    dW[:,:,:,c] += None
                    db[:,:,:,c] += None
                    
        # Установить i-й обучающий пример dA_prev в недобавленный da_prev_pad (Подсказка: X[pad:-pad, pad:-pad, :])
        dA_prev[i, :, :, :] = None
    ### ОКОНЧАНИЕ ВАШЕГО КОДА ЗДЕСЬ ###
    
    # Проверка корректности размера
    assert(dA_prev.shape == (m, n_H_prev, n_W_prev, n_C_prev))
    
    return dA_prev, dW, db

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(10,4,4,3)
W = np.random.randn(2,2,3,8)
b = np.random.randn(1,1,1,8)
hparameters = {"pad" : 2,
               "stride": 2}
Z, cache_conv = conv_forward(A_prev, W, b, hparameters)

# Тестирование conv_backward
dA, dW, db = conv_backward(Z, cache_conv)
print("dA_mean =", np.mean(dA))
print("dW_mean =", np.mean(dW))
print("db_mean =", np.mean(db))

**Ожидаемый выход:**
<table>
    <tr>
        <td>
            **dA_mean**
        </td>
        <td>
            1.45243777754
        </td>
    </tr>
    <tr>
        <td>
            **dW_mean**
        </td>
        <td>
            1.72699145831
        </td>
    </tr>
    <tr>
        <td>
            **db_mean**
        </td>
        <td>
            7.83923256462
        </td>
    </tr>

</table>


## 5.2 Pooling слой - обратное распространение

Далее, необходимо реализовать обратный проход для pooling-слоя, начиная со слоя MAX-POOL. Даже несмотря на то, что pooling слой не имеет параметров для обновления backprop, все равно нужно выполнить обратный переход через pooling слой, чтобы вычислить градиенты для слоев, которые пришли до pooling слоя. 

### 5.2.1 Max pooling - обратное распространение

Перед тем, как перейти к обратной разметке pooling, необходимо построить вспомогательную функцию `create_mask_from_window()`, которая делает следующее: 

$$ X = \begin{bmatrix}
1 && 3 \\
4 && 2
\end{bmatrix} \quad \rightarrow  \quad M =\begin{bmatrix}
0 && 0 \\
1 && 0
\end{bmatrix}\tag{4}$$

Как видно, эта функция создает матрицу "mask", которая отслеживает, где находится максимум матрицы. True (1) указывает на позицию максимума в X, остальные записи - False (0). 

**Упражнение**: Реализуйте `create_mask_from_window()`. 
Подсказки:
- [np.max()]() - вычисляет максимум в массиве.
- Если у вас есть матрица X и скаляр x: `A = (X == x)` вернет матрицу А того же размера, что и X:
```
A[i,j] = True if X[i,j] = x
A[i,j] = False if X[i,j] != x
```
- Здесь не нужно рассматривать случаи, когда в матрице есть несколько максимумов.

In [ ]:
def create_mask_from_window(x):
    """
    Создает маску из входной матрицы x, чтобы определить максимальный вход x.
    
    Arguments:
    x -- Array размером (f, f)
    
    Returns:
    mask --  Array той же формы, что и окно, содержит True в позиции, соответствующей максимальному вводу x.
    """
    
    ### НАЧАЛО ВАШЕГО КОДА ЗДЕСЬ ### (≈1 строка кода)
    mask = None
    ### ОКОНЧАНИЕ ВАШЕГО КОДА ЗДЕСЬ ###
    
    return mask

In [ ]:
np.random.seed(1)
x = np.random.randn(2,3)
mask = create_mask_from_window(x)
print('x = ', x)
print("mask = ", mask)

**Ожидаемый выход:** 

<table> 
<tr> 
<td>

**x =**
</td>

<td>

[[ 1.62434536 -0.61175641 -0.52817175] <br>
 [-1.07296862  0.86540763 -2.3015387 ]]

  </td>
</tr>

<tr> 
<td>
**mask =**
</td>
<td>
[[ True False False] <br>
 [False False False]]
</td>
</tr>


</table>

Why do we keep track of the position of the max? It's because this is the input value that ultimately influenced the output, and therefore the cost. Backprop is computing gradients with respect to the cost, so anything that influences the ultimate cost should have a non-zero gradient. So, backprop will "propagate" the gradient back to this particular input value that had influenced the cost. 

### 5.2.2 - Average pooling - обратное распространение

При max pooling, для каждого окна, все "влияние" на выход поступало от одного входного значения - максимум. В average pooling каждый элемент окна входа имеет равное влияние на выход. Поэтому для реализации backprop необходимо реализовать вспомогательную функцию, которая отражает это.

Например, если сделать average pooling в прямом проходе с помощью фильтра 2x2, то маска, которую необходимо использовать для обратного прохода, будет выглядеть так: 
$$ dZ = 1 \quad \rightarrow  \quad dZ =\begin{bmatrix}
1/4 && 1/4 \\
1/4 && 1/4
\end{bmatrix}\tag{5}$$

Это означает, что каждая позиция в матрице $dZ$ одинаково способствует выходу, потому что в прямом проходе использовалось среднее. 

**Упражнение**: Выполните функцию, описанную ниже, чтобы равномерно распределить значение dz по матрице нужной формы. [Подсказка](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ones.html)

In [13]:
def distribute_value(dz, shape):
    """
    Распределяет входное значение в матрице до необходимо размера
    
    Arguments:
    dz -- входной скаляр
    shape -- размер (n_H, n_W) выходной матрицы, для которой распределяется значение dz.
    
    Returns:
    a -- Array размером (n_H, n_W) для распределено значение dz
    """
    
    ### НАЧАЛО ВАШЕГО КОДА ЗДЕСЬ ###
    # Извлечение размеров (≈1 строка кода)
    (n_H, n_W) = None
    
    # Вычислить значение для распределения по матрице (≈1 строка кода)
    average = None
    
    # Создать матрицу, в которой каждая запись будет представлять собой "average" значение. (≈1 строка кода)
    a = None
    ### ОКОНЧАНИЕ ВАШЕГО КОДА ЗДЕСЬ ###
    
    return a

In [ ]:
a = distribute_value(2, (2,2))
print('distributed value =', a)

**Ожидаемый выход**: 

<table> 
<tr> 
<td>
distributed_value =
</td>
<td>
[[ 0.5  0.5]
<br\> 
[ 0.5  0.5]]
</td>
</tr>
</table>

### 5.2.3 Складываем вместе: Pooling обратное распространение

Теперь у вас есть все, что нужно для расчета обратного распространения на уровне пула.

**Упражнение**: 
Реализуйте функцию `pool_backward` в обоих режимах (`max"` и `average"`). Необходимо использовать 4 for-loops (итерации по тренировочным примерам, высоте, ширине и каналам). Необходимо использовать оператор `if/elif`, чтобы увидеть, равен ли режим `'max'` или `'average'`. Если она равна 'average', необходимо использовать функцию `distribute_value()`, которая реализована выше, для создания матрицы той же формы, что и `a_slice`. В противном случае режим будет равен '`max`', и необходимо создадите маску с `create_mask_from_window()` и умножить ее на соответствующее значение dA.

In [12]:
def pool_backward(dA, cache, mode = "max"):
    """
    Реализация обратного распространения для пулингового слоя
    
    Arguments:
    dA -- градиент стоимости относительно выхода слоя-pool, такая же форма, как и у A
    cache -- cache вывод из прямого прохода слоя-pool, содержит входные значения слоя и hparameters     
    mode -- "max" или "average"
    
    Returns:
    dA_prev -- градиент стоимости по отношению к pooling, та же форма, что и A_prev
    """
    
    ### НАЧАЛО ВАШЕГО КОДА ЗДЕСЬ ###
    
    # Извлечение параметров cache (≈1 строка кода)
    (A_prev, hparameters) = None
    
    # Извлечение гиперпараметров из "hparameters" (≈2 строки кода)
    stride = None
    f = None
    
    # Размер A_prev и dA (≈2 строки кода)
    m, n_H_prev, n_W_prev, n_C_prev = None
    m, n_H, n_W, n_C = None
    
    # Инициализация dA_prev нулями (≈1 строка кода)
    dA_prev = None
    
    for i in range(None):                       # цикл по обучающим примерам
        
        # Выбор обучающего примера из A_prev (≈1 строка кода)
        a_prev = None
        
        for h in range(None):                   # цикл по вертикальной оси
            for w in range(None):               # цикл по горизонтальной оси
                for c in range(None):           # цикл по каналам
                    vert_start = None
                    vert_end = None
                    horiz_start = None
                    horiz_end = None                    
                    if mode == "max":
                        # Используйте углы и "c", чтобы определить текущий срез из a_prev. (≈1 строка кода)
                        a_prev_slice = None
                        # Создайте mask из a_prev_slice (≈1 строка кода)
                        mask = None
                        # Установите dA_prev = dA_prev + (маска, умноженная на правильную запись dA) (≈1 строка кода)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += None
                        
                    elif mode == "average":
                        # Вычисление dA (≈1 строка кода)
                        da = None
                        # Размер фильтра fxf (≈1 строка кода)
                        shape = None
                        # Добавление da с правильным срезом к dA_prev (≈1 строка кода)
                        dA_prev[i, vert_start: vert_end, horiz_start: horiz_end, c] += None
                        
    ### END CODE ###
    
    # Проверка размера
    assert(dA_prev.shape == A_prev.shape)
    
    return dA_prev

In [ ]:
np.random.seed(1)
A_prev = np.random.randn(5, 5, 3, 2)
hparameters = {"stride" : 1, "f": 2}
A, cache = pool_forward(A_prev, hparameters)
dA = np.random.randn(5, 4, 2, 2)

dA_prev = pool_backward(dA, cache, mode = "max")
print("mode = max")
print('mean of dA = ', np.mean(dA))
print('dA_prev[1,1] = ', dA_prev[1,1])  
print()
dA_prev = pool_backward(dA, cache, mode = "average")
print("mode = average")
print('mean of dA = ', np.mean(dA))
print('dA_prev[1,1] = ', dA_prev[1,1]) 

**Ожидаемый выход**: 

mode = max:
<table> 
<tr> 
<td>

**mean of dA =**
</td>

<td>

0.145713902729

  </td>
</tr>

<tr> 
<td>
**dA_prev[1,1] =** 
</td>
<td>
[[ 0.          0.        ] <br>
 [ 5.05844394 -1.68282702] <br>
 [ 0.          0.        ]]
</td>
</tr>
</table>

mode = average
<table> 
<tr> 
<td>

**mean of dA =**
</td>

<td>

0.145713902729

  </td>
</tr>

<tr> 
<td>
**dA_prev[1,1] =** 
</td>
<td>
[[ 0.08485462  0.2787552 ] <br>
 [ 1.26461098 -0.25749373] <br>
 [ 1.17975636 -0.53624893]]
</td>
</tr>
</table>